In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas
import os

In [2]:
import pandas as pd
df = pd.read_csv("/home/pqbas/projects/LABINM_Robotics_Automation/SeedlingsNet/classifiers/data/gallery_31_03_23/Plantines_31_03_23 - Sheet1.csv")

n_cells_wihtout_seedlings = (df[:][:] == -1).sum().sum()
total_cells = df.size


print('% percentage_cells:', ((total_cells - n_cells_wihtout_seedlings)/total_cells)*100)

df

% percentage_cells: 59.72222222222222


,col1,col2,col3,col4,col5,col6
row1,-1.00,-1.00,-1.00,74.42,96.15,-1.00
row2,-1.00,-1.00,63.38,79.07,72.99,-1.00
row3,-1.00,90.86,-1.00,53.29,85.81,86.12
row4,74.64,63.99,94.18,-1.00,69.66,-1.00
row5,-1.00,87.60,94.44,59.42,-1.00,81.19
row6,100.08,65.64,-1.00,86.62,78.90,-1.00
row7,88.31,-1.00,-1.00,89.06,76.34,64.40
row8,-1.00,-1.00,-1.00,40.24,78.51,-1.00
row9,1.00,83.04,59.72,-1.00,68.34,91.16
row10,-1.00,79.30,-1.00,71.37,42.66,34.90


In [18]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas
import os

def get_bbox_from_mask(mask):

    seg_value = 1

    if mask is not None:
        np_seg = np.array(mask)
        segmentation = np.where(np_seg == seg_value)

        # Bounding Box
        bbox = 0, 0, 0, 0
        if len(segmentation) != 0 and len(segmentation[1]) != 0 and len(segmentation[0]) != 0:
            x_min = int(np.min(segmentation[1]))
            x_max = int(np.max(segmentation[1]))
            y_min = int(np.min(segmentation[0]))
            y_max = int(np.max(segmentation[0]))
            bbox = x_min, y_min, x_max, y_max
            return bbox
        
        return None
    else:
        # Handle error case where segmentation image cannot be read or is empty
        print("Error: Segmentation image could not be read or is empty.")
        return None


def get_data_from_gallery(path):

    try:
        mask = plt.imread(path)
    except FileNotFoundError:
        #print(f"Warning: File not found - {path}")
        return None

    x_min, y_min, x_max, y_max = get_bbox_from_mask(mask)
    if x_min is None or y_min is None or x_max is None or y_max is None:
        # Si no se puede calcular la bounding box, retorna None
        #print(f"Warning: Unable to calculate bounding box for - {path}")
        return None
    
    height = y_max - y_min
    width = x_max - x_min
    area = cv2.countNonZero(mask)

    return {
        'heigth':height,
        'width':width,
        'area':area,
    }


def find_csv_files(directory):
    """
    # Specify the directory you want to search in
    directory_to_search = "../gallery_03_03_23_tray1"
    
    # Call the function to find CSV files in the directory
    csv_files_found = find_csv_files(directory_to_search) 

    if csv_files_found:
        print("CSV files found:")
        for csv_file in csv_files_found:
            print(csv_file)
    else:
        print("No CSV files found in the directory:", directory_to_search)

    """
    csv_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".csv"):
                csv_files.append(os.path.join(root, file))
    return csv_files

In [37]:
folders_to_include = [
    "gallery_03_03_23_tray1",
    "gallery_03_03_23_tray2",
    "gallery_17_03_23",
    "gallery_24_03_23",
    "gallery_31_03_23",
]

dataset = {
    'length':[],
    'vertical_area': [],
    'vertical_heigth': [],
    'vertical_width': [],
    'horizontal_area': [],
    'horizontal_heigth': [],
    'horizontal_width': [],
    'vertical_mask_path': [],
    'horizontal_mask_path': [],
    'vertical_rgb_path': [],
    'horizontal_rgb_path': [],
    'vertical_depth_path': [],
    'horizontal_depth_path': []
}

for folder in folders_to_include:
    folder_path = f"../{folder}/"

    csv_file = find_csv_files(folder_path)
    
    if csv_file:
        df = pd.read_csv(csv_file[0])
    

    for x in range(1,13):
        for y in range(1,7):
            
            for image_type in ['vertical', 'horizontal']:
                
                path = f"{folder_path}{image_type}/mask/seedlings_mask_{x}_{y}.jpg"
                data = get_data_from_gallery(path)
                
                if data:
                    dataset[f'{image_type}_area'].append(data['area'])
                    dataset[f'{image_type}_width'].append(data['width'])
                    dataset[f'{image_type}_heigth'].append(data['heigth'])
                    dataset[f'{image_type}_mask_path'].append(path)
            
            
                path = f"{folder_path}{image_type}/rgb/seedlings_{x}_{y}.jpg"  
                if data:
                    dataset[f'{image_type}_rgb_path'].append(path)

                path = f"{folder_path}{image_type}/depth/seedlings_{x}_{y}.jpg"  
                if data:
                    dataset[f'{image_type}_depth_path'].append(path)

            if data != None:
                length = df[f'col{y}'][f'row{x}']
                dataset[f'length'].append(length)

            

# print(len(dataset['length']))

dataframe = pandas.DataFrame(dataset)
dataframe.to_csv('dataseed.csv', index=False)

dataframe

,length,vertical_area,vertical_heigth,vertical_width,horizontal_area,horizontal_heigth,horizontal_width,vertical_mask_path,horizontal_mask_path,vertical_rgb_path,horizontal_rgb_path,vertical_depth_path,horizontal_depth_path
0,77.80,6284,119,79,6390,150,87,../gallery_03_03_23_tray1/vertical/mask/seedli...,../gallery_03_03_23_tray1/horizontal/mask/seed...,../gallery_03_03_23_tray1/vertical/rgb/seedlin...,../gallery_03_03_23_tray1/horizontal/rgb/seedl...,../gallery_03_03_23_tray1/vertical/depth/seedl...,../gallery_03_03_23_tray1/horizontal/depth/see...
1,63.66,14398,167,151,5332,127,85,../gallery_03_03_23_tray1/vertical/mask/seedli...,../gallery_03_03_23_tray1/horizontal/mask/seed...,../gallery_03_03_23_tray1/vertical/rgb/seedlin...,../gallery_03_03_23_tray1/horizontal/rgb/seedl...,../gallery_03_03_23_tray1/vertical/depth/seedl...,../gallery_03_03_23_tray1/horizontal/depth/see...
2,70.85,13696,159,159,6085,143,95,../gallery_03_03_23_tray1/vertical/mask/seedli...,../gallery_03_03_23_tray1/horizontal/mask/seed...,../gallery_03_03_23_tray1/vertical/rgb/seedlin...,../gallery_03_03_23_tray1/horizontal/rgb/seedl...,../gallery_03_03_23_tray1/vertical/depth/seedl...,../gallery_03_03_23_tray1/horizontal/depth/see...
3,59.36,12697,167,143,5123,119,79,../gallery_03_03_23_tray1/vertical/mask/seedli...,../gallery_03_03_23_tray1/horizontal/mask/seed...,../gallery_03_03_23_tray1/vertical/rgb/seedlin...,../gallery_03_03_23_tray1/horizontal/rgb/seedl...,../gallery_03_03_23_tray1/vertical/depth/seedl...,../gallery_03_03_23_tray1/horizontal/depth/see...
4,65.51,13367,127,207,5366,119,119,../gallery_03_03_23_tray1/vertical/mask/seedli...,../gallery_03_03_23_tray1/horizontal/mask/seed...,../gallery_03_03_23_tray1/vertical/rgb/seedlin...,../gallery_03_03_23_tray1/horizontal/rgb/seedl...,../gallery_03_03_23_tray1/vertical/depth/seedl...,../gallery_03_03_23_tray1/horizontal/depth/see...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,58.31,10404,159,119,4300,103,63,../gallery_31_03_23/vertical/mask/seedlings_ma...,../gallery_31_03_23/horizontal/mask/seedlings_...,../gallery_31_03_23/vertical/rgb/seedlings_11_...,../gallery_31_03_23/horizontal/rgb/seedlings_1...,../gallery_31_03_23/vertical/depth/seedlings_1...,../gallery_31_03_23/horizontal/depth/seedlings...
248,67.39,13690,159,231,5605,111,119,../gallery_31_03_23/vertical/mask/seedlings_ma...,../gallery_31_03_23/horizontal/mask/seedlings_...,../gallery_31_03_23/vertical/rgb/seedlings_11_...,../gallery_31_03_23/horizontal/rgb/seedlings_1...,../gallery_31_03_23/vertical/depth/seedlings_1...,../gallery_31_03_23/horizontal/depth/seedlings...
249,45.63,7286,103,135,3894,95,71,../gallery_31_03_23/vertical/mask/seedlings_ma...,../gallery_31_03_23/horizontal/mask/seedlings_...,../gallery_31_03_23/vertical/rgb/seedlings_12_...,../gallery_31_03_23/horizontal/rgb/seedlings_1...,../gallery_31_03_23/vertical/depth/seedlings_1...,../gallery_31_03_23/horizontal/depth/seedlings...
250,33.64,3035,63,87,3724,95,63,../gallery_31_03_23/vertical/mask/seedlings_ma...,../gallery_31_03_23/horizontal/mask/seedlings_...,../gallery_31_03_23/vertical/rgb/seedlings_12_...,../gallery_31_03_23/horizontal/rgb/seedlings_1...,../gallery_31_03_23/vertical/depth/seedlings_1...,../gallery_31_03_23/horizontal/depth/seedlings...


In [7]:

import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas
import os
import os.path
import shutil
import uuid

def get_bbox_from_mask(mask):

    seg_value = 1

    if mask is not None:
        np_seg = np.array(mask)
        segmentation = np.where(np_seg == seg_value)

        # Bounding Box
        bbox = 0, 0, 0, 0
        if len(segmentation) != 0 and len(segmentation[1]) != 0 and len(segmentation[0]) != 0:
            x_min = int(np.min(segmentation[1]))
            x_max = int(np.max(segmentation[1]))
            y_min = int(np.min(segmentation[0]))
            y_max = int(np.max(segmentation[0]))
            bbox = x_min, y_min, x_max, y_max
            return bbox
        
        return None
    else:
        # Handle error case where segmentation image cannot be read or is empty
        print("Error: Segmentation image could not be read or is empty.")
        return None


def get_data_from_gallery(path):

    try:
        mask = plt.imread(path)
    except FileNotFoundError:
        print(f"Warning: File not found - {path}")
        return None

    x_min, y_min, x_max, y_max = get_bbox_from_mask(mask)
    if x_min is None or y_min is None or x_max is None or y_max is None:
        # Si no se puede calcular la bounding box, retorna None
        print(f"Warning: Unable to calculate bounding box for - {path}")
        return None
    
    height = y_max - y_min
    width = x_max - x_min
    area = cv2.countNonZero(mask)

    return {
        'heigth':height,
        'width':width,
        'area':area,
    }

def verificar_images(folder_path, x, y):
    for image_type in ['vertical', 'horizontal']:
        image_path = f"{folder_path}{image_type}/mask/seedlings_mask_{x}_{y}.jpg"
        
        try:
            ver = get_data_from_gallery(image_path)
            return ver
        
        except FileNotFoundError:
            continue

def copy_and_rename(src_folder, dst_folder, unique_name):
    if not os.path.exists(dst_folder):
        print(f"Error: La carpeta de destino {dst_folder} no existe.")
        exit()

    src = os.path.join(src_folder)
    dst = os.path.join(dst_folder, unique_name)
    shutil.copy(src, dst)
    return unique_name


# def copy_image(gallery_path, dataset_path):

#     # for folder_name in gallery_path:
#     # folder_total = f"{folder_name}"

#     ruta_origen_A = f"{gallery_path}/horizontal/rgb"
#     ruta_origen_B = f"{gallery_path}/vertical/rgb"

#     ruta_destino_C = f"{dataset_path}/horizontal/rgb"
#     ruta_destino_D = f"{dataset_path}/vertical/rgb"

#     print(f"Copying and renaming from {ruta_origen_A} to {ruta_destino_C}...")
#     copy_and_rename(ruta_origen_A, ruta_destino_C)

#     print(f"Copying and renaming from {ruta_origen_B} to {ruta_destino_D}...")
#     copy_and_rename(ruta_origen_B, ruta_destino_D)
#     print("Completed Process.")


def almacenar_datos(folder_path, dataset_path, x, y):
    for image_type in ['vertical', 'horizontal']:
        path = f"{folder_path}{image_type}/mask/seedlings_mask_{x}_{y}.jpg"
        data = get_data_from_gallery(path)
        
        if data:
            dataset[f'{image_type}_area'].append(data['area'])
            dataset[f'{image_type}_width'].append(data['width'])
            dataset[f'{image_type}_heigth'].append(data['heigth'])

            unique_name = f"{str(uuid.uuid4())[:8]}.jpg"
            
            unique_name = copy_and_rename(
                os.path.join(folder_path, f"{image_type}/rgb/seedlings_{x}_{y}.jpg"),
                os.path.join(f"{dataset_path}/images", f"{image_type}/rgb"),
                unique_name
            )
            rgb_path = os.path.join(f"images", f"{image_type}/rgb/{unique_name}")

            dataset[f'{image_type}_rgb_path'].append(rgb_path)



            unique_name = copy_and_rename(
                os.path.join(folder_path, f"{image_type}/mask/seedlings_mask_{x}_{y}.jpg"),
                os.path.join(f"{dataset_path}/images", f"{image_type}/mask"),
                unique_name
            )
            mask_path = os.path.join(f"images", f"{image_type}/mask/{unique_name}")

            dataset[f'{image_type}_mask_path'].append(mask_path)



            unique_name = copy_and_rename(
                os.path.join(folder_path, f"{image_type}/depth/seedlings_{x}_{y}.jpg"),
                os.path.join(f"{dataset_path}/images", f"{image_type}/depth"),
                unique_name
            )
            depth_path = os.path.join(f"images", f"{image_type}/depth/{unique_name}")

            dataset[f'{image_type}_depth_path'].append(depth_path)


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas
import os
import os.path
import shutil
import uuid

folders_to_include = [
    "../gallery/gallery_03_03_23_tray1",
    "../gallery/gallery_03_03_23_tray2",
    "../gallery/gallery_17_03_23",
    "../gallery/gallery_24_03_23",
    "../gallery/gallery_31_03_23"
]


dataset = {
    'vertical_area': [],
    'vertical_heigth': [],
    'vertical_width': [],
    'horizontal_area': [],
    'horizontal_heigth': [],
    'horizontal_width': [],
    'vertical_mask_path': [],
    'horizontal_mask_path': [],
    'vertical_rgb_path': [],
    'horizontal_rgb_path': [],
    'vertical_depth_path': [],
    'horizontal_depth_path': []
}


dataset_path = "../dataset"

for folder in folders_to_include:
    folder_path = f"../{folder}/"

    for x in range(1,13):
        for y in range(1,7):
            
            if verificar_images(folder_path, x, y) is False:
                continue
            almacenar_datos(folder_path, dataset_path, x, y)

dataframe = pandas.DataFrame(dataset)
dataframe.to_csv('../dataset/data.csv', index=False)
dataframe

,vertical_area,vertical_heigth,vertical_width,horizontal_area,horizontal_heigth,horizontal_width,vertical_mask_path,horizontal_mask_path,vertical_rgb_path,horizontal_rgb_path,vertical_depth_path,horizontal_depth_path
0,6284,119,79,6390,150,87,images/vertical/mask/9001af37.jpg,images/horizontal/mask/321c1bdc.jpg,images/vertical/rgb/9001af37.jpg,images/horizontal/rgb/321c1bdc.jpg,images/vertical/depth/9001af37.jpg,images/horizontal/depth/321c1bdc.jpg
1,14398,167,151,5332,127,85,images/vertical/mask/046d13e4.jpg,images/horizontal/mask/8d6f75f4.jpg,images/vertical/rgb/046d13e4.jpg,images/horizontal/rgb/8d6f75f4.jpg,images/vertical/depth/046d13e4.jpg,images/horizontal/depth/8d6f75f4.jpg
2,13696,159,159,6085,143,95,images/vertical/mask/195020dc.jpg,images/horizontal/mask/37aaf3df.jpg,images/vertical/rgb/195020dc.jpg,images/horizontal/rgb/37aaf3df.jpg,images/vertical/depth/195020dc.jpg,images/horizontal/depth/37aaf3df.jpg
3,12697,167,143,5123,119,79,images/vertical/mask/09eb74c5.jpg,images/horizontal/mask/47500282.jpg,images/vertical/rgb/09eb74c5.jpg,images/horizontal/rgb/47500282.jpg,images/vertical/depth/09eb74c5.jpg,images/horizontal/depth/47500282.jpg
4,13367,127,207,5366,119,119,images/vertical/mask/910d0b4b.jpg,images/horizontal/mask/c13f3d30.jpg,images/vertical/rgb/910d0b4b.jpg,images/horizontal/rgb/c13f3d30.jpg,images/vertical/depth/910d0b4b.jpg,images/horizontal/depth/c13f3d30.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
247,10404,159,119,4300,103,63,images/vertical/mask/d3efbf15.jpg,images/horizontal/mask/49b9b447.jpg,images/vertical/rgb/d3efbf15.jpg,images/horizontal/rgb/49b9b447.jpg,images/vertical/depth/d3efbf15.jpg,images/horizontal/depth/49b9b447.jpg
248,13690,159,231,5605,111,119,images/vertical/mask/35bf6f7d.jpg,images/horizontal/mask/ddc7cda0.jpg,images/vertical/rgb/35bf6f7d.jpg,images/horizontal/rgb/ddc7cda0.jpg,images/vertical/depth/35bf6f7d.jpg,images/horizontal/depth/ddc7cda0.jpg
249,7286,103,135,3894,95,71,images/vertical/mask/917a8a0b.jpg,images/horizontal/mask/39d6d065.jpg,images/vertical/rgb/917a8a0b.jpg,images/horizontal/rgb/39d6d065.jpg,images/vertical/depth/917a8a0b.jpg,images/horizontal/depth/39d6d065.jpg
250,3035,63,87,3724,95,63,images/vertical/mask/bf034355.jpg,images/horizontal/mask/cffdb272.jpg,images/vertical/rgb/bf034355.jpg,images/horizontal/rgb/cffdb272.jpg,images/vertical/depth/bf034355.jpg,images/horizontal/depth/cffdb272.jpg
